In [ ]:
import hepfile as hf
import fastq as fq

In [ ]:
filepath = '1_control_18S_2019_minq7.fastq'
#filepath = 'fastq-test-input.fastq'

In [ ]:
fastq = fq.read(filepath)
fos = list(fastq)

In [ ]:
import cProfile
with cProfile.Profile() as profile:
    hepfile = hf.initialize()
    for fo in fos:
        meta = fo.getHead().split()

        # get the sample id, read, and ch as group name b/c I think that is unique 
        group = f"{meta[2].split('=')[-1]}_{meta[3].split('=')[-1]}_{meta[4].split('=')[-1]}"

        # create the group and add metadata for that group
        hf.create_group(hepfile, group, counter=f'n_{group}')
        hf.add_meta(hepfile, group, meta)

        # create a dataset underneath that group
        hf.create_dataset(hepfile, 'sequence', group=group, dtype=str)
        hf.create_dataset(hepfile, 'quality', group=group, dtype=str)
        hf.add_meta(hepfile, f'{group}/sequence', 'units')

    bucket = hf.create_single_bucket(hepfile)
    for fo in fos:

        meta = fo.getHead().split()
        group = f"{meta[2].split('=')[-1]}_{meta[3].split('=')[-1]}_{meta[4].split('=')[-1]}"

        seq = list(fo.getSeq())
        qual = list(fo.getQual())
        bucket[f'{group}/sequence'] = seq
        bucket[f'{group}/quality'] = qual

    return_value = hf.pack(hepfile,bucket,STRICT_CHECKING=True,verbose=False)  
    
    profile.print_stats()

In [ ]:
outfile = 'fastq-test.h5'
hf.write_to_file(outfile, hepfile)

In [ ]:
import cProfile
with cProfile.Profile() as profile:
    awk, bucket = hf.load(outfile) #, return_awkward=True)
    profile.print_stats()